In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import keras

# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential, Model, load_model
# from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
#                           BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)
# from keras.callbacks import ModelCheckpoint
# from keras import metrics
# from keras.optimizers import Adam 
# from keras import backend as K
# from keras.losses import binary_crossentropy, categorical_crossentropy
# from keras.utils import Sequence, to_categorical
# from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

import matplotlib.pyplot as plt
import PIL
import cv2
import tensorflow as tf
import pandas as pd
# import imgaug as ia
import numpy as np

# from imgaug import augmenters as iaa

from PIL import Image, ImageOps
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score

%config InlineBackend.figure_format="svg"
%matplotlib inline

ModuleNotFoundError: No module named 'cv2'

In [ ]:
adam = tf.keras.optimizers.Adam()

<p>train data</p>

In [ ]:
df_train=pd.read_csv('random_train.csv')

In [ ]:
df_train=df_train.drop(labels='Unnamed: 0',axis=1)

In [ ]:
df_train.head(n=6)

In [ ]:
df_train.shape

In [ ]:
x=df_train['Pathes']
y=df_train['Diagnosis']

In [ ]:
x,y=shuffle(x,y,random_state=8)

In [ ]:
y.hist()

<p style="color:green"><b>[new try]</b></p>

In [ ]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

In [ ]:
df_train,df_validate=train_test_split(df_train, test_size=0.15,random_state=8)

In [ ]:
IMAGE_SIZE    = (299, 299)
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 20

In [ ]:
df_train.shape

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
train_batches = train_datagen.flow_from_dataframe(dataframe=df_train,
                                                  directory=None,
                                                  x_col="Pathes",
                                                  y_col="Diagnosis",
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='raw',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

In [ ]:

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
valid_batches = valid_datagen.flow_from_dataframe(dataframe=df_validate,
                                                  directory=None,
                                                  x_col="Pathes",
                                                  y_col="Diagnosis",
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='raw',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

In [ ]:
valid_datagen.size

In [ ]:
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))


In [ ]:
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)


In [ ]:
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True


In [ ]:
from tensorflow.keras.optimizers import Adam
net_final.compile(optimizer=Adam(lr=1e-2),
                  loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

<p style="color:blue">saving the model and weights</p>

In [ ]:
model_json = net_final.to_json()
with open("inception_resnet_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

<p style="color:red"><b>loading the trained model and predict. not related. just in case</b></p>

In [ ]:
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
